# Algorytmy tekstowe - laboraorium 2

# 1. Zadanie polega na implementacji dwóch algorytmów kompresji:

### 1. statycznego algorytmu Huffmana (2 p)

In [3]:
from queue import PriorityQueue
from collections import Counter

class Node:
    def __init__(self, frequency, sign=None):
        self.frequency = frequency
        self.sign = sign
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency


def build_tree(signs):
    frequencies = Counter(signs)
    nodes = [Node(frequencies[sign], sign) for sign in frequencies]

    pq = PriorityQueue()
    for node in nodes:
        pq.put(node)
    
    root = None
    while True:
        left = pq.get()
        right = pq.get()
        parent = Node(left.frequency + right.frequency)
        parent.left, parent.right = left, right
        pq.put(parent)
        
        if pq.qsize() == 1: # root
            root = pq.get()
            break

    return root


def build_code_table(node, codes = {}, code = ""):
    if node.sign is not None:
        codes[node.sign] = code
        
    if node.left is not None:
        build_huffman_code_table(node.left, codes, code + "0")
    if node.right is not None:
        build_huffman_code_table(node.right, codes, code + "1")

    return codes

def static_huffman(data):
    root = build_tree(data)
    codes = build_code_table(root)
    
    result = ""
    
    for sign in data:
        result += codes[sign]
        
    return result


### 2. dynamicznego algorytmu Huffmana (3 p)

In [ ]:
class Node:
    def __init__(self, symbol=None, weight=0, parent=None, left=None, right=None):
        self.symbol = symbol
        self.weight = weight
        self.parent = parent
        self.left = left
        self.right = right

class AdaptiveHuffmanEncoder:
    def __init__(self):
        self.root = Node()
        self.symbol_nodes = {}

    def encode(self, data):
        bits = []
        for symbol in data:
            if symbol not in self.symbol_nodes:
                node = Node(symbol=symbol, weight=1)
                self.symbol_nodes[symbol] = node
                self.update_tree(node)
            node = self.symbol_nodes[symbol]
            bits.extend(self.get_code(node))
            node.weight += 1
            self.update_tree(node)
        return ''.join(bits)

    def update_tree(self, node):
        parent = node.parent
        if parent is None:
            return
        nodes = sorted(parent.left, parent.right, key=lambda n: (n.weight, id(n)))
        if nodes[0] is not node:
            nodes[0], nodes[1] = nodes[1], nodes[0]
        self.update_tree(parent)

    def get_code(self, node):
        code = []
        while node.parent is not None:
            if node == node.parent.left:
                code.append('0')
            else:
                code.append('1')
            node = node.parent
        return reversed(code)